In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from fuzzywuzzy import fuzz

import sys
sys.path.append('../src/modules')
import recommendations as rec

In [2]:
df = pd.read_csv('../data/movie.csv')
df.head(5)

,id,title,year_release,duration,genre,rating,metascore,director,cast,votes,description,img_url
0,tt0439572,The Flash,2023,144 min,"Action, Adventure, Fantasy",7.2,56.0,Andy Muschietti,"Ezra Miller,Michael Keaton,Sasha Calle,Michael...","66,700",Barry Allen uses his super speed to change the...,https://m.media-amazon.com/images/M/MV5BZWE2ZW...
1,tt9362722,Spider-Man: Across the Spider-Verse,2023,140 min,"Animation, Action, Adventure",8.9,86.0,Joaquim Dos Santos,"Kemp Powers,Justin K. Thompson,Shameik Moore,H...","151,801","Miles Morales catapults across the Multiverse,...",https://m.media-amazon.com/images/M/MV5BMzI0Nm...
2,tt5090568,Transformers: Rise of the Beasts,2023,127 min,"Action, Adventure, Sci-Fi",6.6,42.0,Steven Caple Jr.,"Anthony Ramos,Dominique Fishback,Luna Lauren V...","28,929","During the '90s, a new faction of Transformers...",https://m.media-amazon.com/images/M/MV5BZTNiND...
3,tt1630029,Avatar: The Way of Water,2022,192 min,"Action, Adventure, Fantasy",7.6,67.0,James Cameron,"Sam Worthington,Zoe Saldana,Sigourney Weaver,S...","416,844",Jake Sully lives with his newfound family form...,https://m.media-amazon.com/images/M/MV5BYjhiNj...
4,tt5971474,The Little Mermaid,I 2023,135 min,"Adventure, Family, Fantasy",7.2,59.0,Rob Marshall,"Halle Bailey,Jonah Hauer-King,Melissa McCarthy...","88,433",A young mermaid makes a deal with a sea witch ...,https://m.media-amazon.com/images/M/MV5BYTUxYj...


In [4]:
sim_mat = sparse.load_npz('../data/npz/similarity_matrix.npz')

In [5]:
# Get Title Similarity Score
def title_matching_score(title_a, title_b):
    return fuzz.ratio(title_a, title_b)

def get_title_from_index(index):
    return df[df.index == index]['title'].values[0]

def get_index_from_title(title):
    return df[df.title == title].index.values[0]

# Get Closest Title
def get_closest_title(title):
    scores = list(enumerate(df['title'].apply(lambda x: title_matching_score(title, x))))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    
    #get the title of the closest match
    closest_title = get_title_from_index(scores[0][0])
    score = scores[0][1]

    return closest_title, score

def get_recommendations(title, n=10, sim_mat=sim_mat):
    closest_title, distance_score = get_closest_title(title)

    if distance_score != 100:
        print("Did you mean " + closest_title + "?")

    title_index = get_index_from_title(closest_title)
    movies_similar = sim_mat[title_index].toarray().flatten()
    movie_indices = np.argsort(-movies_similar)[1:n + 1]
    movie_scores = movies_similar[movie_indices]

    print("Top " + str(n) + " similar movies to " + closest_title + " are:\n")
    for idx, movie_index in enumerate(movie_indices):
        movie_title = get_title_from_index(movie_index)
        print(f"{movie_title} (Similarity Score: {movie_scores[idx]})")

    # return list(zip(movie_indices, movie_scores))
    return list(movie_indices)

In [9]:
rec.get_recommendations('Grumpy gumpy', df, sim_mat)

Did you mean Grumpy Old Men?


,id,title,year_release,duration,genre,rating,metascore,director,cast,votes,description,img_url
6829,tt0077889,Magic,1978,107 min,"Drama, Horror",6.8,49.0,Richard Attenborough,"Anthony Hopkins,Ann-Margret,Burgess Meredith,E...","12,337",A ventriloquist is at the mercy of his vicious...,https://m.media-amazon.com/images/M/MV5BZjQ5NT...
6619,tt0113228,Grumpier Old Men,1995,101 min,"Comedy, Romance",6.6,46.0,Howard Deutch,"Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ...","28,753",John and Max resolve to save their beloved bai...,https://m.media-amazon.com/images/M/MV5BMjQxM2...
5433,tt0063374,The Odd Couple,1968,105 min,Comedy,7.6,86.0,Gene Saks,"Jack Lemmon,Walter Matthau,John Fiedler,Herb E...","36,275",A New Yorker newly separated from his wife mov...,https://m.media-amazon.com/images/M/MV5BZDVhNz...
8022,tt0071524,The Front Page,1974,105 min,"Comedy, Drama, Romance",7.3,62.0,Billy Wilder,"Jack Lemmon,Walter Matthau,Susan Sarandon,Vinc...","14,426",A ruthless editor tries to get his top reporte...,https://m.media-amazon.com/images/M/MV5BMDYxMz...
2731,tt0066892,Carnal Knowledge,1971,98 min,"Comedy, Drama",6.9,77.0,Mike Nichols,"Jack Nicholson,Candice Bergen,Art Garfunkel,An...","14,151",Chronicling the lifelong sexual development of...,https://m.media-amazon.com/images/M/MV5BNTA5N2...
1777,tt0102138,JFK,1991,189 min,"Drama, History, Thriller",8.0,72.0,Oliver Stone,"Kevin Costner,Gary Oldman,Jack Lemmon,Walter M...","162,444",New Orleans District Attorney Jim Garrison dis...,https://m.media-amazon.com/images/M/MV5BYjYyOD...
5400,tt0074258,Burnt Offerings,1976,116 min,"Horror, Mystery, Thriller",6.4,48.0,Dan Curtis,"Karen Black,Oliver Reed,Burgess Meredith,Eilee...","12,706",A family moves into a large old mansion in the...,https://m.media-amazon.com/images/M/MV5BMjEzNz...
2210,tt2568862,Going in Style,2017,96 min,"Comedy, Crime",6.6,50.0,Zach Braff,"Michael Caine,Alan Arkin,Ann-Margret,Matt Dillon","88,557",Desperate to pay the bills and come through fo...,https://m.media-amazon.com/images/M/MV5BMTc5Mz...
5777,tt0077699,House Calls,1978,98 min,"Comedy, Drama, Romance",6.6,49.0,Howard Zieff,"Walter Matthau,Glenda Jackson,Art Carney,Richa...","1,956",Charley is a surgeon who's recently lost his w...,https://m.media-amazon.com/images/M/MV5BMWI5Nz...
6834,tt0095736,A Night in the Life of Jimmy Reardon,1988,90 min,"Comedy, Drama, Romance",5.5,NaN,William Richert,"River Phoenix,Ann Magnuson,Meredith Salenger,I...","2,935",A charming womanizer has to find a way to get ...,https://m.media-amazon.com/images/M/MV5BMTc3YW...
